In [ ]:
# # @title Install madrona_mjx and quadruped_mjx_rl

# # install additional missing packages
# !sudo apt install -y libx11-dev libxrandr-dev libxinerama-dev libxcursor-dev libxi-dev mesa-common-dev

# # install jax with cuda enabled
!pip install "jax[cuda12]==0.5.1"

# # get madrona mjx and its subpackages
# !mkdir modules
# !git clone https://github.com/shacklettbp/madrona_mjx.git modules/madrona_mjx
# !git -C modules/madrona_mjx submodule update --init --recursive

# # build madrona mjx
# !mkdir modules/madrona_mjx/build
# !cd modules/madrona_mjx/build && cmake -DLOAD_VULKAN=OFF .. && make -j 8

# # install madrona mjx
# !pip install -e modules/madrona_mjx

# Clones and installs our Quadruped RL package
!git clone https://github.com/alexeiplatzer/unitree-go2-mjx-rl.git
!pip install -e unitree-go2-mjx-rl